Import libraries used for this shit.

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import random
import re

from collections import Counter, OrderedDict
from typing import Dict, Iterable, List, Optional

Load the dataset.

In [7]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
valid_data = pd.read_csv('data/valid.csv')

Only take data from source: 'OpenSubtitles v2018'.

In [87]:
train_data = train_data[train_data['source'] == 'OpenSubtitles v2018']
test_data = test_data[test_data['source'] == 'OpenSubtitles v2018']
valid_data = valid_data[valid_data['source'] == 'OpenSubtitles v2018']

In [111]:
train_data_sampled = train_data.sample(n=500000, random_state=42)

Take the first 2 columns from the dataset (english and vietnamese, respectively).

In [112]:
train_en = train_data_sampled['en']
train_vi = train_data_sampled['vi']

test_en = test_data['en']
test_vi = test_data['vi']

valid_en = valid_data['en']
valid_vi = valid_data['vi']

Just checking.

In [113]:
print(train_en.shape)

(500000,)


In [114]:
train_en.head()

2269779                       There are thousands of glens.
2674860                               I pushed you in that.
2541945                 So he wouldn't lie or use an alias.
1865205    I remember identifying you after the exhumation.
831567              This is where you've been for 20 years?
Name: en, dtype: object

## Tokenizers

A tokenizer is used to turn a string into a list of tokens that make up that string, e.g. "good morning!" becomes ["good", "morning", "!"]. We'll start talking about the sentences being a sequence of tokens from now, instead of saying they're a sequence of words. What's the difference? Well, "good" and "morning" are both words and tokens, but "!" is not a word. We could say "!" is punctuation, but the term token is more general and covers: words, punctuation, numbers and any special symbols.

This function helps with special suffices.

In [115]:
def split_suffix(word):
    # Check for "n't" suffix (negative contractions)
    if re.search(r"n't$", word):
        return [word]  # Return the whole word as a single token
    
    # Check for specific suffixes ('d, 've, 's, 'm, 'll, 're)
    match = re.search(r"(.*)('d|'ve|'s|'m|'ll|'re)$", word)
    if match:
        return [match.group(1), match.group(2)]  # Return the part before and the suffix

    # Check for a word ending with just '
    match = re.search(r"(.*)('$)$", word)
    if match:
        return [match.group(1), match.group(2)]  # Return the part before and just '

    # Check for apostrophe + other characters (return <unk>)
    if re.search(r"(.*)('\w+)$", word):
        return ["<unk>"]  # Return <unk>

    return [word]  # Return the word unchanged if no match

The following function tokenizes a line of text.

In [116]:
patterns = [r"\"", r"\.", r"<br \/>", r",", r"\(", r"\)", r"\!", r"\?", r"\;", r"\:", r"\s+"]
replacements = ["", " . ", " ", " , ", " ( ", " ) ", " ! ", " ? ", " ", " ", " "]
patterns_dict = list((re.compile(p), r) for p, r in zip(patterns, replacements))

def get_tokenizer_line(line):
    """
    Basic normalization for a line of text.
    Normalization includes
    - lowercasing
    - complete some basic text normalization for English words as follows:
        add spaces before and after '\''
        remove '\"',
        add spaces before and after '.'
        replace '<br \/>'with single space
        add spaces before and after ','
        add spaces before and after '('
        add spaces before and after ')'
        add spaces before and after '!'
        add spaces before and after '?'
        replace ';' with single space
        replace ':' with single space
        replace multiple spaces with single space

    Returns a list of tokens after splitting on whitespace.
    """
    line = line.lower()
    for pattern_re, replaced_str in patterns_dict:
        line = pattern_re.sub(replaced_str, line)
    result = [split_suffix(word) for word in line.split()]
    flattened_result = [token for sublist in result for token in sublist]
    return flattened_result

print(get_tokenizer_line("You can now install TorchText (but I can't) and this's for checking the split_suffix functions' i'dk!"))

['you', 'can', 'now', 'install', 'torchtext', '(', 'but', 'i', "can't", ')', 'and', 'this', "'s", 'for', 'checking', 'the', 'split_suffix', 'functions', "'", '<unk>', '!']


The following function tokenizes text (in general).

In [117]:
def get_tokenizer(text):
    tokens = []
    for line in text:
        tokens.append(get_tokenizer_line(line))
    return tokens
        
get_tokenizer(["You can now install TorchText using pip!", "This is a test."])

[['you', 'can', 'now', 'install', 'torchtext', 'using', 'pip', '!'],
 ['this', 'is', 'a', 'test', '.']]

This function tokenizes the dataset.

In [118]:
def tokenize_example(example, max_length, sos_token, eos_token):
    """
    Apply the tokenizer to all of the examples in each data split, as well as some other processing.

    This function takes in an example from the dataset, applies the get_tokenizer function, trims the list of tokens to a maximum length, 
    and then appends the start of sequence and end of sequence tokens to the beginning and end of the list of tokens.
    """
    en_tokens = get_tokenizer(example["en"])[:max_length]
    vi_tokens = get_tokenizer(example["vi"])[:max_length]
    en_tokens = [[sos_token] + token + [eos_token] for token in en_tokens]
    vi_tokens = [[sos_token] + token + [eos_token] for token in vi_tokens]
    return {"en_tokens": en_tokens, "vi_tokens": vi_tokens}

In [119]:
tokenize_example(train_data_sampled[:5], 20, "<sos>", "<eos>")

{'en_tokens': [['<sos>',
   'there',
   'are',
   'thousands',
   'of',
   'glens',
   '.',
   '<eos>'],
  ['<sos>', 'i', 'pushed', 'you', 'in', 'that', '.', '<eos>'],
  ['<sos>',
   'so',
   'he',
   "wouldn't",
   'lie',
   'or',
   'use',
   'an',
   'alias',
   '.',
   '<eos>'],
  ['<sos>',
   'i',
   'remember',
   'identifying',
   'you',
   'after',
   'the',
   'exhumation',
   '.',
   '<eos>'],
  ['<sos>',
   'this',
   'is',
   'where',
   'you',
   "'ve",
   'been',
   'for',
   '20',
   'years',
   '?',
   '<eos>']],
 'vi_tokens': [['<sos>',
   'có',
   'hàng',
   'ngàn',
   'thung',
   'lũng',
   'như',
   'thế',
   '.',
   '<eos>'],
  ['<sos>', 'tớ', 'đẩy', 'cậu', 'đi', 'bằng', 'cái', 'đó', '.', '<eos>'],
  ['<sos>',
   'nên',
   'hắn',
   'sẽ',
   'không',
   'nói',
   'dối',
   'hay',
   'dùng',
   'bí',
   'danh',
   '.',
   '<eos>'],
  ['<sos>',
   'anh',
   'nhớ',
   'lại',
   'lúc',
   'nhận',
   'dạng',
   'em',
   'khi',
   'họ',
   'đào',
   'mộ',
   '.',
   '<eo

Here, we're trimming all sequences to a maximum length of 1000000000 tokens, and using *sos* and *eos* as the start and end of sequence tokens, respectively.

In [120]:
max_length = 1_000_000_000
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "max_length": max_length,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data_tokenized = tokenize_example(train_data_sampled, **fn_kwargs)
valid_data_tokenized = tokenize_example(valid_data, **fn_kwargs)
test_data_tokenized = tokenize_example(test_data, **fn_kwargs)

In [121]:
print(len(train_data_tokenized["en_tokens"]))

500000


We can now look at some examples, confirming the two new features have been added; both of which are lowercased list of strings with the start/end of sequence tokens appended.

In [122]:
print(test_data_tokenized)

{'en_tokens': [['<sos>', 'four', 'years', ',', 'and', 'you', 'never', 'talked', 'to', 'her', '.', '<eos>'], ['<sos>', 'the', 'man', 'that', "'s", 'giving', 'the', 'test', 'has', 'serious', 'doubts', 'that', 'they', "'ll", 'get', 'an', 'accurate', 'result', '.', '<eos>'], ['<sos>', 'mmm', '.', 'the', 'constitutional', 'lawyer', '?', '<eos>'], ['<sos>', 'because', 'if', 'you', "can't", 'remember', 'where', 'that', 'silo', 'was', ',', 'you', "'re", 'worthless', 'to', 'us', '.', '<eos>'], ['<sos>', 'only', 'relevant', 'if', 'it', 'can', 'cause', 'pulmonary', 'edema', 'and', 'delirium', '.', '<eos>'], ['<sos>', 'why', "didn't", 'she', 'escape', '?', '<eos>'], ['<sos>', 'what', 'kind', 'of', 'game', 'is', 'he', 'playing', '?', '<eos>'], ['<sos>', 'i', 'was', 'just', 'telling', 'these', 'guys', 'about', 'this', 'job', 'that', 'i', 'did', 'in', 'paris', 'for', 'a', 'couple', 'of', 'months', ',', 'and', 'this', 'guy', ',', 'he', 'let', 'me', 'smoke', 'some', 'of', 'this', 'stuff', '.', '<eos>']

## Vocabularies

Next, we'll build the vocabulary for the source and target languages. The vocabulary is used to associate each unique token in our dataset with an index (an integer), e.g. "hello" = 1, "world" = 2, "bye" = 3, "hates" = 4, etc. When feeding text data to our model, we convert the string into tokens and then the tokens into numbers using the vocabulary as a look up table, e.g. "hello world" becomes ["hello", "world"] which becomes [1, 2] using the example indices given. We do this as neural networks cannot operate on strings, only numerical values.

In theory, our vocabulary can be large enough to have an index for every unique token in our dataset. However, what happens if a token exists in our validation and test set, but not in our training set? In that case, we replace the token with an "unknown token", denoted by **_unk_**, which is given its own index (usually index zero). All unknown tokens are replaced by **_unk_**, even if the tokens are different, i.e. if the tokens "gilgamesh" and "enkidu" were both not within our vocabulary, then the string "gilgamesh hates enkidu" gets tokenized to ["gilgamesh", "hates", "enkidu"] and then becomes [0, 24, 0] (where "hates" has the index 24).

Ideally, we want our model to be able to handle unknown tokens by learning to use the context around them to make translations. The only way it can learn that is if we also have unknown tokens in the training set. Hence, when creating our vocabularies with build_vocab_from_iterator, we use the "min_freq" to not create an index for tokens which appear less than min_freq times in our training set. In other words, when using the vocabulary, any token which does not appear at least twice in our training set will get replaced by the unknown token index when converting tokens to indices.

We also use the "specials" argument to pass special tokens. These are tokens which we want to add to the vocabulary but do not necessarily appear in our tokenized examples. These special tokens will appear first in the vocabulary. We've already discussed the unk_token, sos_token, and eos_token. The final special token is the pad_token, denoted by **_pad_**.

When inputting sentences into our model, it is more efficient to pass multiple sentences at once (known as a batch), instead of one at a time. The requirement for sentences to be batched together is that they all have to be the same length (in terms of the number of tokens). The majority of our sentences are not the same length, but we can solve this by "padding" (adding **_pad_**> tokens) the tokenized version of each sentence in a batch until they all have equal tokens to the longest sentence in the batch. For example, if we had two sentences: "I love pizza" and "I hate music videos". They would be tokenized to something like: ["i", "love", "pizza"] and ["i", "hate", "music", "videos"]. The first sequence of tokens would then be padded to ["i", "love", "pizza", "**_pad_**"]. Both sequences could then be converted to indexes using the vocabulary.

That was a lot of information, but torchtext can handle all the fuss of building the vocabulary. Unfortunately, I can't import torchtext, so the whole process will have to be built manually (Đm torchtext).

In [103]:
class Vocabulary:
    def __init__(self, max_vocab_size=None, min_freq=1, specials=['<pad>', '<unk>', '<bos>', '<eos>']):
        self.max_vocab_size = max_vocab_size
        self.min_freq = min_freq
        self.specials = specials
        self.token2idx = {}
        self.idx2token = []
        self.counter = Counter()

        # Adding special tokens first
        self.add_special_tokens()

    def add_special_tokens(self):
        for token in self.specials:
            self.add_token(token)

    def add_token(self, token):
        if token not in self.token2idx:
            idx = len(self.idx2token)
            self.token2idx[token] = idx
            self.idx2token.append(token)

    def build_vocab(self, sentences):
        # Count frequencies of all tokens
        for sentence in sentences:
            self.counter.update(sentence)
        
        # Filter out tokens by frequency and limit vocab size
        most_common = self.counter.most_common(self.max_vocab_size) if self.max_vocab_size else self.counter.items()
        for token, freq in most_common:
            if freq >= self.min_freq and token not in self.token2idx:
                self.add_token(token)

    def __len__(self):
        return len(self.idx2token)

    def get_index(self, token):
        return self.token2idx.get(token, self.token2idx['<unk>'])

    def get_token(self, index):
        return self.idx2token[index] if index < len(self.idx2token) else '<unk>'

    def encode(self, sentence):
        return [self.get_index(token) for token in sentence]

    def decode(self, indices):
        return [self.get_token(index) for index in indices]


In [123]:
# Initialize vocabulary
en_vocab = Vocabulary(max_vocab_size=1_000_000_000, min_freq=1)
vi_vocab = Vocabulary(max_vocab_size=1_000_000_000, min_freq=1)

# Build the vocabulary from sentences
en_vocab.build_vocab(train_data_tokenized["en_tokens"])
vi_vocab.build_vocab(train_data_tokenized["vi_tokens"])

print((len(en_vocab), len(vi_vocab)))


(71014, 41633)


In [124]:
# Encode a sentence
encoded_sentence = en_vocab.encode(['<bos>', "i", "love", "learning", "and", "i", "am", "gay", "!", ".", "<eos>"])
print("Encoded:", encoded_sentence)

# Decode the sentence
decoded_sentence = en_vocab.decode(encoded_sentence)
print("Decoded:", decoded_sentence)


Encoded: [2, 8, 161, 2300, 15, 8, 156, 2153, 44, 4, 3]
Decoded: ['<bos>', 'i', 'love', 'learning', 'and', 'i', 'am', 'gay', '!', '.', '<eos>']


In [125]:
print(en_vocab.get_index("i"))

8
